In [1]:
import os
from glob import glob
import arcpy
from arcpy import env

In [2]:
### Step1: identify total patches

os.chdir('')
env.workspace = ''

for root,dirs,files in os.walk('patches_raw'):
    for f in files:
        print(os.path.join(root, f))
        
        # future
        if len(f.split('_')) == 4:
            depth = f.split('_')[1]
            scen = f.split('_')[2]
            fname = f.split('.tif')[0]
            fpath = os.path.join(root, f)
            
            # convert to polygon
            arcpy.ddd.Int(fpath, "step1_patches_id/%s/%s/int_%s" % (depth, scen, f))
            arcpy.conversion.RasterToPolygon("step1_patches_id/%s/%s/int_%s" % (depth, scen, f), 
                                             "step1_patches_id/%s/%s/%s.shp" % (depth, scen, fname), 
                                             "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

            # add field
            arcpy.management.AddField("step1_patches_id/%s/%s/%s.shp" % (depth, scen, fname), 
                                      "pid", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
            arcpy.management.CalculateField("step1_patches_id/%s/%s/%s.shp" % (depth, scen, fname), 
                                            "pid", "!FID! + 1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

            # convert to raster
            with arcpy.EnvManager(extent="-180 -90 180 90"):
                arcpy.conversion.FeatureToRaster("step1_patches_id/%s/%s/%s.shp" % (depth, scen, fname), "pid", 
                                                 "step1_patches_id2rs/%s/%s/%s" % (depth, scen, f), 1)
        # present
        else:
            depth = f.split('_')[1]
            fname = f.split('.tif')[0]
            fpath = os.path.join(root, f)
            
            # convert to polygon
            arcpy.ddd.Int(fpath, "step1_patches_id/%s/int_%s" % (depth, f))
            arcpy.conversion.RasterToPolygon("step1_patches_id/%s/int_%s" % (depth, f), 
                                             "step1_patches_id/%s/%s.shp" % (depth, fname), 
                                             "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

            # add field
            arcpy.management.AddField("step1_patches_id/%s/%s.shp" % (depth, fname), 
                                      "pid", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
            arcpy.management.CalculateField("step1_patches_id/%s/%s.shp" % (depth, fname), 
                                            "pid", "!FID! + 1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

            # convert to raster
            with arcpy.EnvManager(extent="-180 -90 180 90"):
                arcpy.conversion.FeatureToRaster("step1_patches_id/%s/%s.shp" % (depth, fname), "pid", 
                                                 "step1_patches_id2rs/%s/%s" % (depth, f), 1)

In [3]:
### Step2: connect boundaries 
### dissolve adjacent patches and calculate shape area (in decimal degree)
### achieved by R

In [4]:
### Step3: eliminate fragmented patches

os.chdir('')
env.workspace = ''

for root,dirs,files in os.walk('step2_patches_newid'):
    for f in files:
        if os.path.splitext(f)[1] == '.shp':
            fname = os.path.splitext(f)[0]
            fpath = os.path.join(root, fname)

            # eliminate area not more than 5 cell size
            e = 1
            arcpy.management.MakeFeatureLayer("%s.shp" % fpath, fname)
            arcpy.management.SelectLayerByAttribute(fname, "NEW_SELECTION", "shape_area <= 5", None)
            arcpy.management.Eliminate(fname, "%s_E_%s.shp" % (fpath, e), "LENGTH", '', None)
            # calculate residual fragments
            cursor = arcpy.SearchCursor("%s_E_%s.shp" % (fpath, e),'"shape_area" <= 5')
            count = 0
            for r in cursor:
                count += 1

            # stop iteration when completed
            for i in range(1, 100):
                arcpy.management.MakeFeatureLayer("%s_E_%s.shp" % (fpath, e), "%s_E_%s" % (fname, e))
                arcpy.management.SelectLayerByAttribute("%s_E_%s" % (fname, e), 
                                                        "NEW_SELECTION", "shape_area <= 5", None)
                arcpy.management.Eliminate("%s_E_%s" % (fname, e), 
                                           "%s_E_%s.shp" % (fpath, (e+1)), "LENGTH", '', None)
                e += 1

                # calculate residual fragments
                cursor = arcpy.SearchCursor("%s_E_%s.shp" % (fpath, e),'"shape_area" <= 5')
                iterc = 0
                for r in cursor:
                    iterc += 1

                # judge if the iteration is completed
                if iterc == count:
                    break
                else:
                    count = iterc
            
            # save the last eliminated shp as new file
            to_path = 'step3_patches_newid_elim' + fpath.split('step2_patches_newid')[1] + '_elim.shp'
            arcpy.management.SelectLayerByAttribute("%s_E_%s" % (fname, e-1), 
                                                    "NEW_SELECTION", "shape_area <= 5", None)
            arcpy.management.Eliminate("%s_E_%s" % (fname, (e-1)), to_path, "LENGTH", '', None)
            
            print(fpath)


In [5]:
### Step4: divide and refine hemispheric patches

os.chdir('')
env.workspace = ''

for root,dirs,files in os.walk('step3_patches_newid_elim'):
    for f in files:
        if os.path.splitext(f)[1] == '.shp':
            depth = f.split('_')[1]
            fname = os.path.splitext(f)[0]
            fpath = os.path.join(root, fname)
    
            # add field
            arcpy.management.AddFields("%s.shp" % fpath, 
                                       "cp_y DOUBLE # # # #;pid SHORT # # # #;N_hemi SHORT # # # #;S_hemi SHORT # # # #")
            # identify patches' ID
            arcpy.management.CalculateField("%s.shp" % fpath, 
                                            "pid", "!FID! + 1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
            # calculate y-centroid
            arcpy.management.CalculateGeometryAttributes("%s.shp" % fpath, 
                                                         "cp_y CENTROID_Y", '', '', None, "SAME_AS_INPUT")
            # pre-divide patches by y-centroid
            arcpy.management.CalculateField("%s.shp" % fpath, "N_hemi", "judge(!cp_y!)", "PYTHON3", 
                                            """def judge(y):
              if y >= -10:
                return 1
              else:
                return 0""", "TEXT", "NO_ENFORCE_DOMAINS")
            arcpy.management.CalculateField("%s.shp" % fpath, "S_hemi", "judge(!cp_y!)", "PYTHON3", 
                                            """def judge(y):
              if y <= 10:
                return 1
              else:
                return 0""", "TEXT", "NO_ENFORCE_DOMAINS")
            
            # export patches by hemisphere
            N_root = "step4_patches_hemi/N_hemi" + root.split('step3_patches_newid_elim')[1]
            S_root = "step4_patches_hemi/S_hemi" + root.split('step3_patches_newid_elim')[1]
            arcpy.conversion.FeatureClassToFeatureClass("%s.shp" % fpath, N_root, 
                                                        "N_%s.shp" % fname, "N_hemi = 1")
            arcpy.conversion.FeatureClassToFeatureClass("%s.shp" % fpath, S_root, 
                                                        "S_%s.shp" % fname, "S_hemi = 1")

            # convert to raster
            arcpy.conversion.FeatureToRaster("%s/N_%s.shp" % (N_root, fname), "pid", 
                                             "%s/N_%s.tif" % (N_root, fname), 1)
            arcpy.conversion.FeatureToRaster("%s/S_%s.shp" % (S_root, fname), "pid", 
                                             "%s/S_%s.tif" % (S_root, fname), 1)

            # project raster
            ## Northern
            N_topath = "step4_patches_hemi/N_hemi_proj" + root.split('step3_patches_newid_elim')[1]
            with arcpy.EnvManager(snapRaster="topo/N_topo/N_%s.tif" % depth):
                arcpy.management.ProjectRaster("%s/N_%s.tif" % (N_root, fname), 
                                               "%s/N_%s.tif" % (N_topath, fname), 
                                               'PROJCS["WGS_1984_Azimuthal_Equidistant",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Azimuthal_Equidistant"],PARAMETER["false_easting",0.0],PARAMETER["false_northing",0.0],PARAMETER["central_meridian",0.0],PARAMETER["latitude_of_origin",90.0],UNIT["Meter",1.0]]', 
                                               "NEAREST", "97300 111000", None, None, 
                                               'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_VERTICAL")
            ## Southern
            S_topath = "step4_patches_hemi/S_hemi_proj" + root.split('step3_patches_newid_elim')[1]
            with arcpy.EnvManager(snapRaster="topo/S_topo/S_%s.tif" % depth):
                arcpy.management.ProjectRaster("%s/S_%s.tif" % (S_root, fname), 
                                               "%s/S_%s.tif" % (S_topath, fname), 
                                               'PROJCS["WGS_1984_Azimuthal_Equidistant",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Azimuthal_Equidistant"],PARAMETER["false_easting",0.0],PARAMETER["false_northing",0.0],PARAMETER["central_meridian",0.0],PARAMETER["latitude_of_origin",-90.0],UNIT["Meter",1.0]]', 
                                               "NEAREST", "97300 111000", None, None, 
                                               'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_VERTICAL")    

            # convert to polygon
            ## Northern
            arcpy.conversion.RasterToPolygon("%s/N_%s.tif" % (N_topath, fname), 
                                             "%s/N_%s.shp" % (N_topath, fname), 
                                             "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)
            ## Southern
            arcpy.conversion.RasterToPolygon("%s/S_%s.tif" % (S_topath, fname), 
                                             "%s/S_%s.shp" % (S_topath, fname), 
                                             "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)
            
            # dissolve multi-parts
            ## Northern
            arcpy.management.Dissolve("%s/N_%s.shp" % (N_topath, fname), "%s/N_%s_dsov.shp" % (N_topath, fname), 
                                      "gridcode", None, "MULTI_PART", "DISSOLVE_LINES")
            ## Southern
            arcpy.management.Dissolve("%s/S_%s.shp" % (S_topath, fname), "%s/S_%s_dsov.shp" % (S_topath, fname), 
                                      "gridcode", None, "MULTI_PART", "DISSOLVE_LINES")
            
            # add field and calculate centroids
            ## Northern
            arcpy.management.AddFields("%s/N_%s_dsov.shp" % (N_topath, fname), "cp_x DOUBLE # # # #;cp_y DOUBLE # # # #")
            arcpy.management.CalculateGeometryAttributes("%s/N_%s_dsov.shp" % (N_topath, fname), 
                                                         "cp_x CENTROID_X;cp_y CENTROID_Y", '', '', None, "SAME_AS_INPUT")
            ## Southern
            arcpy.management.AddFields("%s/S_%s_dsov.shp" % (S_topath, fname), "cp_x DOUBLE # # # #;cp_y DOUBLE # # # #")
            arcpy.management.CalculateGeometryAttributes("%s/S_%s_dsov.shp" % (S_topath, fname), 
                                                         "cp_x CENTROID_X;cp_y CENTROID_Y", '', '', None, "SAME_AS_INPUT")
            
            # create centroids points from x-y table 
            ## Northern
            N_centro = "step4_patches_hemi/N_centro" + root.split('step3_patches_newid_elim')[1]
            arcpy.management.XYTableToPoint("%s/N_%s_dsov.shp" % (N_topath, fname), "%s/N_%s_centro.shp" % (N_centro, fname), 
                                            "cp_x", "cp_y", None, 
                                            'PROJCS["WGS_1984_Azimuthal_Equidistant",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Azimuthal_Equidistant"],PARAMETER["false_easting",0.0],PARAMETER["false_northing",0.0],PARAMETER["central_meridian",0.0],PARAMETER["latitude_of_origin",90.0],UNIT["Meter",1.0]];-20004100 -20004100 225133828.933593;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision')
            ## Southern
            S_centro = "step4_patches_hemi/S_centro" + root.split('step3_patches_newid_elim')[1]
            arcpy.management.XYTableToPoint("%s/S_%s_dsov.shp" % (S_topath, fname), "%s/S_%s_centro.shp" % (S_centro, fname), 
                                            "cp_x", "cp_y", None, 
                                            'PROJCS["WGS_1984_Azimuthal_Equidistant",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Azimuthal_Equidistant"],PARAMETER["false_easting",0.0],PARAMETER["false_northing",0.0],PARAMETER["central_meridian",0.0],PARAMETER["latitude_of_origin",-90.0],UNIT["Meter",1.0]];-20004100 -20004100 225133828.933593;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision')
            
            print(f)


In [6]:
### Step5: snap point to nearest cell center
### calculate Euclidean distance between centroid point and other raster cell centers, snap with the nearest
### export as shapefile points
### achieved by R